In [1]:
from llama_index.llms.ollama import Ollama

In [13]:
from pathlib import Path
import os

from llama_index.vector_stores.chroma import ChromaVectorStore

In [8]:
cwd = Path(os.getcwd()).parent
os.chdir(cwd)

In [9]:
def get_ollama_LLM(model=None):
    model = model or "phi4:latest"
    context_window = 1000

    ollama_llm = Ollama(
        model=model,
        request_timeout=120.0,
        context_window=context_window,
        )
    return ollama_llm

In [19]:
from src.embedding_client import RemoteEmbedding
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

In [14]:
vector_store = ChromaVectorStore(
    collection_name="hot_pot_qa_v2",
    host="localhost",
    port=8010
)

In [27]:
embed_model = RemoteEmbedding(f"http://localhost:8020")
llm = get_ollama_LLM()

In [30]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model, llm=llm)
retriever = index.as_retriever(similarity_top_k=3)

In [38]:
query_engine = index.as_query_engine(streaming=False)

In [58]:
query_engine.__dict__

{'_retriever': <llama_index.core.indices.vector_store.retrievers.retriever.VectorIndexRetriever at 0x7f6333bdcd90>,
 '_response_synthesizer': <llama_index.core.response_synthesizers.compact_and_refine.CompactAndRefine at 0x7f63355eff10>,
 '_node_postprocessors': [],
 'callback_manager': <llama_index.core.callbacks.base.CallbackManager at 0x7f6335896d40>}

In [41]:
x = query_engine.query('Who directed the movie A Kiss for Corliss?')

In [47]:
for i in (x.__dict__)['source_nodes']:
    print(i)

Node ID: 5a8c7595554299585d9e36b6_0
Text: A Kiss for Corliss is a 1949 American comedy film directed by
Richard Wallace and written by Howard Dimsdale. It stars Shirley
Temple in her final starring role as well as her final film
appearance. It is a sequel to the 1945 film "Kiss and Tell". "A Kiss
for Corliss" was retitled "Almost a Bride" before release and this
title appears in the tit...
Score:  0.607

Node ID: 5ae1b1445542997283cd223d_4
Text: A Kiss for Corliss is a 1949 American comedy film directed by
Richard Wallace and written by Howard Dimsdale. It stars Shirley
Temple in her final starring role as well as her final film
appearance. It is a sequel to the 1945 film "Kiss and Tell". "A Kiss
for Corliss" was retitled "Almost a Bride" before release and this
title appears in the tit...
Score:  0.607



In [56]:
(i.__dict__)['node'].__dict__#['metadata']

{'id_': '5ae1b1445542997283cd223d_4',
 'embedding': None,
 'metadata': {'question': 'Who starred in her final film role in the 1964 film directed by the man who also did Vera Cruz and Kiss Me Deadly?',
  'type': 'bridge',
  'answer': 'Mary Astor',
  'level': 'hard',
  'supporting_facts': 'Robert Aldrich||Hush… Hush, Sweet Charlotte',
  'title': 'A Kiss for Corliss'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text': 'A Kiss for Corliss is a 1949 American comedy film directed by Richard Wallace and written by Howard Dimsdale. It stars Shirley Temple in her final starring role as well as her final film appearance. It is a sequel to the 1945 film "Kiss and Tell". "A Kiss for Corliss" was retitled "Almost a Bride" before release and this title appears in the title sequence. The film was released on November 25, 1949, by United Artists.',
 'mimetype': 'text/plain',
 'sta

In [31]:
from llama_index.core.agent.workflow import ReActAgent

In [39]:
from llama_index.core.tools import RetrieverTool
#retriever_tool = RetrieverTool.from_retriever(retriever, name="Query Retriever")

In [59]:
from llama_index.core.tools import QueryEngineTool

query_engine_tool = QueryEngineTool.from_defaults(
    query_engine, name="Vector DB Search", description="This tool allows search to vector database"
)
agent = ReActAgent(tools = [query_engine_tool], llm=llm)

In [60]:
agent.run('Has Richard Wallace directed comedy movies ?')

<WorkflowHandler pending cb=[Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-b907b01ff7d4', bound_args=<BoundArgumen...StartEvent())>, instance=ReActAgent(na...SER: 'user'>)), context=<_contextvars...x7f63313e64c0>)() at /root/.cache/pypoetry/virtualenvs/chat-bot-ui-v2-E1yGBO_D-py3.10/lib/python3.10/site-packages/llama_index_instrumentation/dispatcher.py:277]>

In [61]:
from src.response_utils import stream_agent_response

In [62]:
prompt = 'Has Richard Wallace directed comedy movies ?'
stream_agent_response(agent, prompt)

<async_generator object stream_agent_response at 0x7f6330f78240>

In [63]:
await stream_agent_response(agent, prompt)

TypeError: object async_generator can't be used in 'await' expression